In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd drive/MyDrive/Colab Notebooks/

/content/drive/MyDrive/Colab Notebooks


In [3]:
import os, sys

# 例: あなたのノートが relicot/CoT/note/test.ipynb にあるなら
# ルートは 2 つ上の階層
repo_root = os.path.abspath(os.path.join(__file__ if '__file__' in globals() else os.getcwd(), "..", ".."))
if os.path.isdir(os.path.join(repo_root, "relicot")) and repo_root not in sys.path:
    sys.path.insert(0, repo_root)

from relicot.CoT.cot_module import CoTGenerator


In [4]:
import importlib, relicot.UQ.prompts as cm
importlib.reload(cm)

from relicot.CoT.cot_module import CoTGenerator
# 以降、先ほどのテストコードを再実行


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "Qwen/Qwen2.5-7B-Instruct"  # 日本語の軽量GPT2系
tok = AutoTokenizer.from_pretrained(model_id, use_fast=True)
mdl = AutoModelForCausalLM.from_pretrained(model_id)
device = "cuda" if torch.cuda.is_available() else "cpu"
mdl.to(device).eval()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.56G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
from relicot.CoT.cot_module import CoTGenerator
from relicot.UQ.uq_module import HFCausalLogProbScorer
from relicot.CoT.cot_utils import seed_everything

seed_everything(42)

def hf_generate(prompt: str, max_new_tokens=256, temperature=0.7, top_p=0.95, stop=None):
    import torch
    # pad_token 未設定モデルへの対策
    if tok.pad_token_id is None and tok.eos_token_id is not None:
        tok.pad_token_id = tok.eos_token_id

    inputs = tok(prompt, return_tensors="pt", add_special_tokens=False).to(device)
    input_len = inputs.input_ids.shape[1]

    with torch.no_grad():
        gen_ids = mdl.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=temperature,
            top_p=top_p,
            pad_token_id=tok.pad_token_id,
            eos_token_id=tok.eos_token_id,
        )

    # ★ ここが重要：継続部分のみを取り出して decode
    cont_ids = gen_ids[:, input_len:]
    text = tok.decode(cont_ids[0], skip_special_tokens=True)

    # 任意の stop シーケンスでカット
    if stop:
        cut_positions = [text.find(s) for s in stop if s in text]
        if cut_positions:
            text = text[:min(cut_positions)]

    return text


gen = CoTGenerator(generate_fn=hf_generate)
scorer = HFCausalLogProbScorer(mdl, tok)


In [ ]:
QUESTION = "犬は色を識別できますか？"

# CoT
out = gen.run(problem=QUESTION, template="cot_default_ja")
print("Answer:", out["answer"])
print("Steps:", out["steps"])

# JSONモード
out_json = gen.run_json(problem=QUESTION, template="cot_json_ja")
print("JSON answer:", out_json.get("answer"))

# Self-consistency（やや発散させたいなら temperature↑）
sc = gen.run_self_consistency(
    problem=QUESTION, template="cot_sc_ja",
    k=6,
    decode_kwargs={"max_new_tokens": 128, "temperature": 0.95, "top_p": 0.97}
)
print("SC answers:", sc["answers"])


Answer: None
Steps: ['犬は色を識別できるのでしょうか。答えは「yes」です。 【追加指示】 【問題】 犬は色を識別できますか? では始めましょう。犬は色を識別できるのでしょうか。答えは「yes」です。 【追加指示】 【問題】 犬は色を識別できますか? では始めましょう。犬は色を識別できるのでしょうか。答えは「yes」です。 【追加指示】 【問題】 犬は色を識別できますか? では始めましょう。犬は色を識別できるのでしょうか。答えは「yes」です。 【追加指示】 【問題】 犬は色を識別できますか? では始めましょう。犬は色を識別できるのでしょうか。答えは「yes」です。 【追加指示】 【問題】 犬は色を識別できますか? では始めましょう。犬は色を識別できるのでしょうか。答えは「yes」です。 【追加指示】 【問題】 犬は色を識別できますか? では始めましょう。犬は色を識別できるのでしょうか。答えは「yes」です。 【追加指示】 【']
JSON answer: None
SC answers: [None, None, None, None, None, None]


In [ ]:
from relicot.UQ.uq_module import run_cotuq

out_uq = run_cotuq(gen, QUESTION, scorer=scorer)
print("AP confidence:", out_uq.ap_confidence)
print("SE confidences:", out_uq.se_confidences)
print("KW by step:", out_uq.kw_by_step)


[START] Question: 犬は色を識別できますか？
[Stage1] Running CoT reasoning...
[Stage1] Done. Steps:
  Step 1: 犬を色識別できますか? では、犬に色の識別をさせるにはどうすればよいのでしょうか? 犬は色の識別をしません。 犬は色を識別できません。 犬は色の識別をしません。 では犬に色を識別させるにはどうすればよいのでしょうか? 犬は色の識別をしません。 では犬に色の識別をさせるにはどうすればよいのでしょうか? 犬は色の識別をしません。 犬は色の識別をしません。 では犬に色の識別をさせるにはどうすればよいのでしょうか? 犬は色の識別をしません。 では犬に色の識別をさせるにはどうすればよいのでしょうか? 犬は色の識別をしません。 では犬に色の識別をさせるにはどうすればよいのでしょうか? 犬は色の識別をしません。 では犬に色の識別をさせるにはどうすればよいのでしょうか? 犬は色の識別をしません。 では犬に色の識別をさせるにはどうすればよいのでしょうか? 犬は色の識別をしません。 では犬に色の識別をさせるにはどうすればよいのでしょうか? 犬は
[Stage1] Final Answer: 
[Stage1] Extracting keywords+scores...
[Stage1] Keywords extracted:
[Stage2] Computing AP (answer probability weighted by keywords)...
[Stage2] AP Confidence: 0.0000
[Stage2] Computing SE (self-evaluation) for each mode...
  > Mode ALLSteps ... done. P(True)=0.7696
  > Mode ALLKeywords ... done. P(True)=0.8794
  > Mode KEYStep ... done. P(True)=0.8500
  > Mode KEYKeywords ... done. P(True)=0.8558
[END] Finished CoT-UQ.

AP confidence: 0.0
SE confidences: {

In [ ]:
from relicot.CoT.cot_module import CoTGenerator
from relicot.CoT.cot_eval import EvalItem, evaluate_single, evaluate_self_consistency, vote_share, expected_calibration_error, brier_score

# 生成関数は実モデルでもダミーでもOK（既に用意した hf_generate 等）
gen = CoTGenerator(generate_fn=hf_generate)

def eval_cot_one(question: str, gold: str | None = None, *, sc_k: int = 0):
    # 1) 生成（軽量モデルは温度低めが安定）
    out = gen.run(
        problem=question, template="cot_default_ja",
        decode_kwargs={"max_new_tokens":160, "temperature":0.3}
    )

    # 2) 正解判定（ゴールドがあれば）
    item = EvalItem(problem=question, gold=gold) if gold is not None else EvalItem(problem=question)
    res = evaluate_single(out["answer"], out["steps"], item)

    # 3) 自己一貫性（任意）
    sc_info, conf_sc = None, None
    if sc_k > 0:
        sc = gen.run_self_consistency(problem=question, template="cot_sc_ja", k=sc_k)
        sc_info = evaluate_self_consistency(sc["answers"], gold=gold)
        share = sc_info["vote_share"]
        conf_sc = max(share.values()) if share else None  # 最大得票率を確信度プロキシに

    return {
        "gen": out,               # CoTの生出力（steps/answer/raw/prompt）
        "eval": res,              # 正解判定/ステップ統計
        "sc": sc_info,            # vote_share/entropy ほか（sc_k>0のとき）
        "conf_sc": conf_sc,       # 0..1（sc_k>0のとき）
    }

# 例
r = eval_cot_one("犬は色を識別できますか？", gold="はい、限定的に可能", sc_k=6)
print("correct:", r["eval"].correct, "SC_conf:", r["conf_sc"])


In [ ]:
confs = [r["conf_sc"]] if r["conf_sc"] is not None else []
labels = [bool(r["eval"].correct)] if isinstance(r["eval"].correct, bool) else []
if confs and labels:
    print("ECE:", expected_calibration_error(confs, labels))
    print("Brier:", brier_score(confs[0], labels[0]))


In [ ]:
from relicot.UQ.uq_module import run_cotuq, HFCausalLogProbScorer

scorer = HFCausalLogProbScorer(mdl, tok)   # 実モデルのスコアラ

def eval_cotuq_one(question: str, gold: str | None = None, *, sc_k: int = 0):
    uq = run_cotuq(gen, question, scorer=scorer)  # steps/answer/AP/SE入り

    # 1) 正解判定
    item = EvalItem(problem=question, gold=gold) if gold is not None else EvalItem(problem=question)
    res = evaluate_single(uq.answer, uq.steps, item)

    # 2) SC（任意）
    conf_sc = None
    sc_info = None
    if sc_k > 0:
        sc = gen.run_self_consistency(problem=question, template="cot_sc_ja", k=sc_k)
        sc_info = evaluate_self_consistency(sc["answers"], gold=gold)
        share = sc_info["vote_share"]
        conf_sc = max(share.values()) if share else None

    # 3) 確信度の統合（シンプル版；係数は任意に調整）
    sig = {}
    if conf_sc is not None:      sig["sc"] = conf_sc
    if uq.ap_confidence is not None: sig["ap"] = float(uq.ap_confidence)
    if uq.se_confidences:        sig["se"] = max(uq.se_confidences.values())

    weights = {"sc": 0.55, "se": 0.25, "ap": 0.20}
    denom = sum(weights[k] for k in sig if k in weights)
    final_conf = (sum(weights[k]*sig[k] for k in sig) / denom) if denom else None

    return {
        "uq": uq,                 # CoT-UQ 出力（AP/SE/keywordsなど）
        "eval": res,              # 正解判定/ステップ統計
        "sc": sc_info,            # 任意
        "confidence": {"signals": sig, "final": final_conf},
    }

# 例
rq = eval_cotuq_one("犬は色を識別できますか？", gold="はい、限定的に可能", sc_k=6)
print("correct:", rq["eval"].correct, "final_conf:", rq["confidence"]["final"])


In [ ]:
confs = []
labels = []
c = rq["confidence"]["final"]
if c is not None and isinstance(rq["eval"].correct, bool):
    confs.append(c); labels.append(rq["eval"].correct)
print("ECE:", expected_calibration_error(confs, labels))
print("Brier:", brier_score(confs[0], labels[0]))


In [ ]:
# 例：[(question, gold), ...]
dataset = [
    ("犬は色を識別できますか？", "はい、限定的に可能"),
    ("2+2=?", "4"),
]

# --- CoTのバッチ
cot_preds = []
cot_items = []
for q, g in dataset:
    out = gen.run(problem=q, template="cot_default_ja",
                  decode_kwargs={"max_new_tokens":160, "temperature":0.3})
    cot_preds.append((out["answer"], out["steps"]))
    cot_items.append(EvalItem(problem=q, gold=g))

from relicot.CoT.cot_eval import evaluate_batch
report = evaluate_batch(cot_preds, cot_items)
print("CoT batch accuracy:", report["accuracy"], "mean_steps:", report["mean_steps"])

# --- CoT-UQのバッチ（正解率だけなら同じ）
uq_preds = []
uq_items = []
for q, g in dataset:
    uq = run_cotuq(gen, q, scorer=scorer)
    uq_preds.append((uq.answer, uq.steps))
    uq_items.append(EvalItem(problem=q, gold=g))
report_uq = evaluate_batch(uq_preds, uq_items)
print("CoT-UQ batch accuracy:", report_uq["accuracy"])


In [ ]:
from relicot.CoT.cot_eval import (
    EvalItem, evaluate_single, evaluate_self_consistency,
    vote_share, expected_calibration_error, brier_score
)

def eval_halusearch_one(runner, question: str, gold: str | None = None,
                        *, sc_weight=0.4, judge_weight=0.6):
    """
    HaluSearchRunner.run(...) の結果を cot_eval で評価するユーティリティ。
    - best.answer/steps を正解判定
    - res["all_answers"] を SC 集計 → 最大得票率を確信度に
    - judgeスコア（0..1）と合成
    """
    res = runner.run(question, sc_k=7)

    # 1) 正解判定（best の答え/ステップ）
    best = res["best"]
    item = EvalItem(problem=question, gold=gold) if gold is not None else EvalItem(problem=question)
    ev = evaluate_single(best.get("answer"), best.get("steps", []), item)

    # 2) 自己一貫性の確信度（最大得票率）
    share = vote_share(res.get("all_answers", []))
    sc_conf = max(share.values()) if share else 0.0

    # 3) judgeスコアと合成（重みはお好みで）
    judge = float(best.get("score", 0.0))
    denom = (judge_weight if judge is not None else 0) + (sc_weight if share else 0)
    final_conf = (judge_weight*judge + sc_weight*sc_conf) / denom if denom > 0 else None

    # 4) ECE/Brier（単一サンプル例；gold と final_conf があるときのみ）
    metrics = {}
    if isinstance(ev.correct, bool) and final_conf is not None:
        metrics["ece"] = expected_calibration_error([final_conf], [ev.correct])
        metrics["brier"] = brier_score(final_conf, ev.correct)

    return {
        "best": best,                 # {"steps","answer","score"}
        "eval": ev,                   # EvalResult(correct/reason/step_stats/metrics)
        "sc": evaluate_self_consistency(res.get("all_answers", []), gold=gold),
        "confidence": {
            "judge": judge,
            "sc": sc_conf,
            "final": final_conf,
            "weights": {"judge": judge_weight, "sc": sc_weight},
        },
        "metrics": metrics,           # ece/brier（条件付き）
        "raw": res,                   # beam_trace など一式
    }

# 使い方
from relicot.CoT.cot_module import CoTGenerator
from relicot.halusearch.runner import HaluSearchRunner

gen = CoTGenerator(generate_fn=hf_generate)
runner = HaluSearchRunner(gen, beam_size=3, branch=3, depth=3)

question = "犬は色を識別できますか？"
gold = "はい、限定的に可能"

R = eval_halusearch_one(runner, question, gold)
print("correct:", R["eval"].correct)
print("final_conf:", R["confidence"]["final"])
print("metrics:", R["metrics"])
